<a href="https://colab.research.google.com/github/guilhermelaviola/BIArchitectureAndBigData/blob/main/Class04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Importing all the necessary libraries:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import spacy
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Setting the file search parameters:
filename = 'Data.xlsx'
search_path = '/content/drive/My Drive/Datasets/Descomplica'

# Searching for the file?
file = None
for root, dirs, files in os.walk(search_path):
    if filename in files:
        file= os.path.join(root, filename)
        break
    df = pd.read_excel(file)

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [ ]:
# Importing and displaying the dataset:
data = pd.ExcelFile(file)
c4 = pd.read_excel(data, 'Class04')
c4.tail()

In [ ]:
# Displaying the dataset columns:
df.columns

In [ ]:
df['Texto da Avaliação'] = df['Texto da Avaliação'].fillna('').astype(str)
concat_text = ' '.join(df['Texto da Avaliação'])
words = concat_text.lower().split()
frequency = Counter(words)
df_frequency = pd.DataFrame(frequency.items(), columns=['palavra', 'frequencia'])
df_frequency.sort_values(by='frequencia', ascending=False, inplace=True)
df_frequency.reset_index(drop=True, inplace=True)
df_frequency

In [ ]:
nlp = spacy.load('pt_core_news_sm')

def find_adjectives(text):
  doc = nlp(text)
  adjectives = [token.text.lower() for token in doc if token.pos_ == 'ADJ']
  return adjectives

In [ ]:
all_adjectives = []

for evaluation in df['Texto da Avaliação'].dropna():
  all_adjectives.extend(find_adjectives(evaluation))
adjectives_frequency = Counter(all_adjectives)
df_adjectives = pd.DataFrame(adjectives_frequency.items(), columns=['Adjetivo', 'Frequencia'])
df_adjectives.sort_values(by='Frequencia', ascending=False, inplace=True)
df_adjectives.reset_index(drop=True, inplace=True)
df_adjectives

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(df_adjectives.set_index('Adjetivo')['Frequencia'])

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()